### The following is an adaptation from Moran & Clark (2011) ["Estimating seed and pollen movement in a monoecious plant: a hierarchical Bayesian approach integrating genetic and ecological data"](http://onlinelibrary.wiley.com/doi/10.1111/j.1365-294X.2011.05019.x/full)

Many of the comments are from the original script, but I've added a few.

I use individuals across plots (UTM coordinates) in the parentage analysis, and use in-plot coordinates (x,y) for out-of-plot probabilities. Other than including chloroplasts and multiprocessing, the script remains largely unchanged.

In [ ]:
source('~/imports.R') 

# step 1: enter data and make prelim calculations

In [ ]:
df = read.csv('~/teakettle/data/moran/all_data.txt',header=T,sep='\t')
head(df)

#### <center> for pub, distinguish some adults (e.g., DBH < 20) as seedling </center>

In [ ]:
bframe1 <- df[which(df[,'seedling'] == 'False'),]
bframe2 <- df[which(df[,'seedling'] == 'True'),]
bframe3 <- bframe2[seq(1,nrow(bframe2),2),c('xutm','yutm')]
bframe3[,'Area'] = rep('1',nrow(bframe3))

In [ ]:
head(bframe1)

In [ ]:
head(bframe2)

In [ ]:
head(bframe3)

In [ ]:
#Then enter or calculate number of loci, adults, etc.
nloci <- 7
nNuclear <- 4
nadult <- nrow(bframe1)/2 #number of adults
noff <- nrow(bframe2)/2 #number of seedlings 
nplot <- nrow(bframe3) #number of plots, not needed in script

In [ ]:
nadult

In [ ]:
noff

In [ ]:
nplot

In [ ]:
#Weighting factor (fecundity, size, etc)
###This needs to be len(w)=nadults because of iopol.func
w <- bframe1[seq(1,nrow(bframe1),2),'dbh_11']
len(w) == nadult

In [ ]:
#make sure no NA in weights
mean(w)

In [ ]:
#Size of mapped area? Coordinates for the corners of the mapped area:
pcor <- rbind(c(100,100),c(-100,100),c(-100,-100),c(100,-100)) 
pcr <- nrow(pcor) #4 corners

In [ ]:
#Because there are two rows of genotypes per individual, it is handy to be able to look at only one at a time:
ba1 <- seq(1,nrow(bframe1)-1,by=2) #first genotype, adult
ba2 <- seq(2,nrow(bframe1),by=2) #regenotype, adult
bo1 <- seq(1,nrow(bframe2)-1,by=2) #rows for first genotype, offspring 
bo2 <- seq(2,nrow(bframe2),by=2) #rows for regenotype, offspring

In [ ]:
#Where are adults and offspring located?
adult.loc <- cbind(bframe1[ba1,'xutm'],bframe1[ba1,'yutm']) #adult locations 
off.loc <- cbind(bframe2[bo1,'xutm'],bframe2[bo1,'yutm']) #Seedling locations

In [ ]:
#Seedling census plot information:
#seedling plot locations, size
plot.loc <- cbind(bframe3[,1],bframe3[,2],bframe3[,3]) #seedling plot locations, not used until graphing
nplot <- nrow(plot.loc) #number of seedling plots
off.plot <- rep(NA,noff); #which plot is each offspring in? This is never used!!!!!
pld <- rep(NA,noff) 
for(k in 1:noff) #I can just use the column from my seedling data to specify the plot.ID
    {
        for(p in 1:nplot)
            {
                if(off.loc[k,1]==plot.loc[p,1] & off.loc[k,2]==plot.loc[p,2])
                    { 
                        off.plot[k] <- p
                    }
            }
    }
pld <- bframe3[,3] #size of plot each offspring is in

In [ ]:
head(plot.loc)

In [ ]:
###What are the genotyping error rates? Example below for nloci = 6:
#genotyping error rates per locus
e1 <- rep(0.05,nloci) #mistyping 
e2 <- rep(0.05,nloci) #dropout

In [ ]:
#dispersal priors - truncated normal distributions
msp <- 253 #seed mean
sdsp <- 1000 # seed sd 
mpp <- 2000 #pollen mean 
sdpp <- 1500 # pollen sd

In [ ]:
#...and these limits on the jump distribution for the Gibbs sampler provide the truncation. 
#jump distribution limits for updating seed and pollen parameters in Gibbs sampler
los <- 10 #lower bound on us
his <- 10000 #upper bound on us
lop <- 1 #lower bound on up
hip <- 15000 #upper bound on up 

In [ ]:
#Calculate allele frequencies, number of alleles:
## identify alleles
alist <- numeric(0) #list of unique alleles
aa <- numeric(0) #list of all observed alleles 
nallele <- rep(0,nloci) #number of alleles observed
aa1 <- c(8,10,12,14,16,17,18) #adult allele1 column
aa2 <- c(9,11,13,15,16,17,18) #adult allele2 column
oa1 <- c(8,10,12,14,16,17,18) #seedling allele1 column
oa2 <- c(9,11,13,15,16,17,18) #seedling allele2 column

In [ ]:
for (t in 1:nloci)
{ 
    #c() = c(1st adult allele of the t'th locus,2nd adult allele at the t'th locus, 
        #1st seedling allele ..., 2nd seedling allele ...)
    q <- sort(c(bframe1[ba1,aa1[t]],bframe1[ba1,aa2[t]],bframe2[bo1,oa1[t]],bframe2[bo1,oa2[t]]), 
              na.last=NA, decreasing = FALSE)
    qq <- q[q != 0] #concatenates list of sorted alleles, excludes missing allelotypes
    alist <- append(alist,list(unique(qq))) #IDs unique alleles from qq, adds them to alist
    aa <- append(aa,list(qq)) 
    nallele[t] <- length(unique(qq))
}

In [ ]:
##calculate allele frequencies 
max.na <- max(nallele)
freq <- matrix(NA,nloci,max.na)
for (y in 1:nloci)
{
    f <- c(rep(0,nallele[y])) 
    for (h in 1:(nallele[y]))
    {
        f[h] <- (length(which(aa[[y]]==alist[[y]][h])))/(length(aa[[y]])) 
    }
    freq[y,c(1:nallele[[y]])] <- f 
}

In [ ]:
#Probability of observing a genotype given allele frequencies
pgen <- numeric(0)
for (l in 1:nloci)
{
    fx <- matrix(freq[l,1:nallele[[l]]],nrow=nallele[[l]],ncol=nallele[[l]],byrow=T)
    fy <- matrix(freq[l,1:nallele[[l]]],nrow=nallele[[l]],ncol=nallele[[l]]) 
    pg <- fx*fy
    pgen <- append(pgen,list(pg)) 
}

In [ ]:
# the originl script has only one gadult matrix, I'm using two to allow chloroplast alleles
    #to only be considered when considering a potential father
#Observed genotypes.
## adult genotypes
ga1 <- bframe1[ba1,aa1] 
ga2 <- bframe1[ba1,aa2]
gadult1 <- matrix(NA,nadult,nloci) 
gadult2 <- gadult1
for (j in 1:nadult)
{ 
    for (k in 1:nloci)
    {
        for (q in 1:nallele[[k]])
        {
            if (ga1[j,k]==alist[[k]][q]) #if the j'th adult's k'th locus's allele#1 is the same as the q'th allele of the k'th locus:
            { 
                gadult1[j,k] <- q          #keep track of which allele is the q'th allele of locus k
            }

            if (ga1[j,k]==0)              #if the j'th adult's k'th locus's allele#1 is missing:
            {
                gadult1[j,k] <- 0         #mark as missing
            }

            if (ga2[j,k]==alist[[k]][q]) 
            {
                gadult2[j,k] <- q 
            }

            if (ga2[j,k]==0) 
            {
                gadult2[j,k] <- 0
            }
        }
    }
}
gadult <- append(list(gadult1),list(gadult2))
gadultF <- gadult
gadult[[1]][,5:7] <- 0
gadult[[2]][,5:7] <- 0

In [ ]:
write.table(gadult[[1]],'~/teakettle/gadult1.txt',sep='\t')

In [ ]:
# again, I am using two gadultr matrices to distinguish fathers
# second genotyping
gar1 <- bframe1[ba2,aa1] 
gar2 <- bframe1[ba2,aa2]
gadultr1 <- matrix(NA,nadult,nloci)
gadultr2 <- gadultr1

for (j in 1:nadult)
{
    for (k in 1:nloci)
    {
        for (q in 1:nallele[[k]])
        {
            if (gar1[j,k] == alist[[k]][q]) 
            {
                gadultr1[j,k] <- q 
            }
            if (gar1[j,k] == 0) 
            {
                gadultr1[j,k] <- 0
            }
            if (gar2[j,k] == alist[[k]][q]) 
            {
                gadultr2[j,k] <- q 
            }
            if (gar2[j,k] == 0)
            {
                gadultr2[j,k] <- 0
            }
        }
    }
}
gadult.r <- append(list(gadultr1),list(gadultr2))
gadultF.r <- gadult.r #father
gadult.r[[1]][,5:7] <- 0 #mother
gadult.r[[2]][,5:7] <- 0 #mother

In [ ]:
##offspring genotypes 
go1 <- bframe2[bo1,oa1] #the rows for the offspring first genotype, the columns for the offsprings first allele of each locus
go2 <- bframe2[bo1,oa2] #the rows for the offspring first genotype, the columns for the offsprings second allele of each locus
goff1 <- matrix(NA,noff,nloci) #a matrix containing the first allele of a particular locus
goff2 <- goff1 #a matrix containing the second allele of a particular locus
for (j in 1:noff)
{
    for (k in 1:nloci)
    {
        for (q in 1:nallele[[k]]) #for the qth allele of the kth locus
        {
            if (go1[j,k] == alist[[k]][q]) 
            {
                goff1[j,k] <- q #put the qth allele into the matrix cell corresponding to the jth offspring's kth locus
            }
            if (go1[j,k] == 0) 
            {
                goff1[j,k] <- 0 #offspring is missing a genotype for the first allele
            }
            if (go2[j,k] == alist[[k]][q]) 
            {
                goff2[j,k] <- q 
            }
            if (go2[j,k] == 0) #offspring is missing a genotype for the second allele
            {
                goff2[j,k] <- 0
            }
        }
    }
}
#a list where the 1st element is a matrix of the first allele, 2nd element is a matrix of the 2nd allele
goff <- append(list(goff1),list(goff2)) 

In [ ]:
# regenotyping
gor1 <- bframe2[bo2,oa1] 
gor2 <- bframe2[bo2,oa2]
goff1r <- matrix(NA,noff,nloci) 
goff2r <- goff1r
for (j in 1:noff)
{
    for (k in 1:nloci)
    {
        for (q in 1:nallele[[k]])
        {
            if (gor1[j,k] == alist[[k]][q])
            { 
                goff1r[j,k] <- q 
            }
            if (gor1[j,k] == 0) 
            {
                goff1r[j,k] <- 0
            }
            if (gor2[j,k] == alist[[k]][q]) 
            {
                goff2r[j,k] <- q 
            }
            if (gor2[j,k] == 0) 
            {
                goff2r[j,k] <- 0
            }
        }
    }
}
goff.r <- append(list(goff1r),list(goff2r))

# Step 2: Functions

In [ ]:
#How far apart are adults, seedlings?
##calculates a distance matrix 
distmat <- function(x1,y1,x2,y2)
{
    xd <- outer(x1,x2,function(x1,x2) (x1 - x2)^2) 
    yd <- outer(y1,y2,function(y1,y2) (y1 - y2)^2) 
    d <- t(sqrt(xd + yd))
    return(d)
}

#The 2D-t kernel (probability per m2 of dispersing a given distance), and a truncated normal distribution:
##2D-t dispersal function
# where d is distance, u is disp param
disp.func <- function(d,u)
    {
        1/(pi*u*(1+(d^2/u))^2)
    }


## truncated normal distribution, where lo is lowerbound and hi is upper bound 
tnorm <- function(n,lo,hi,mu,sig)
{
    z <- runif(n,0,1)
    qlo <- pnorm((lo-mu)/sig)
    qhi <- pnorm((hi-mu)/sig)
    rr <- mu + sig*qnorm(qlo+z*(qhi-qlo)) 
    return(rr)
}

#If you want to calculate an expected dispersal distance from a u, or vice versa:
#function to find a u with a mean distance 
mean.func <- function(u.in)
    { 
        pi*(sqrt(u.in))/2 
    }

#function to find a mean distance with a u 
u.func <- function(m.in)
    { 
        (2*m.in/pi)^2 
    }

#Probability of observed genotype (Go) given true genotype (G). Mistyping involves one step in either direction:
pgo.func <- function(a1,a2,e1,e2,l)
{
    gmat <- matrix(0,nallele[l],nallele[l]) 
    PE <- rep(0,8)
    #event 1 - no error
    PE[1] <- ((1-e1[l])^2)*((1-e2[l])^2)/(1-e2[l]^2)
    #event 2 - a1 mistyped
    PE[2] <-e1[l]*(1-e1[l])*((1-e2[l])^2)/(1-e2[l]^2)
    #event 3 - a2 mistyped
    PE[3] <-e1[l]*(1-e1[l])*((1-e2[l])^2)/(1-e2[l]^2)
    #event 4 - both mistyped
    PE[4] <-(e1[l]^2)*((1-e2[l])^2)/(1-e2[l]^2)
    #event 5 - a1 dropped (either correct or mistyped), a2 correct 
    PE[5] <-(1-e1[l])*e2[l]*(1-e2[l])/(1-e2[l]^2)
    #event 6 - a1 correct, a2 dropped (either correct or mistyped) 
    PE[6] <-(1-e1[l])*e2[l]*(1-e2[l])/(1-e2[l]^2)
    #event 7 - a1 dropped (either correct or mistyped), a2 mistyped 
    PE[7] <-e1[l]*(1-e2[l])*e2[l]/(1-e2[l]^2)
    #event 8 - a1 mistyped, a2 dropped (either correct or mistyped) 
    PE[8] <-e1[l]*(1-e2[l])*e2[l]/(1-e2[l]^2)
    # event 1
    gmat[a1,a2] <- PE[1]*1 + gmat[a1,a2]
    #event 5
    gmat[a2,a2] <- PE[5]*1/2 + gmat[a2,a2]
    #event 6
    gmat[a1,a1] <- PE[6]*1/2 + gmat[a1,a1]
    lmax <- nallele[l]-1
    mt1 <- 1/(lmax)
    # event 2
    if(a1>1)
    {
        gmat[1:(a1-1),a2] <- PE[2]*mt1 + gmat[1:(a1-1),a2]
    }
    if(a1<nallele[l]) 
    {
        gmat[(a1+1):nallele[l],a2] <- PE[2]*mt1 + gmat[(a1+1):nallele[l],a2]
    }
    # event 3
    if(a2>1) 
    {
        gmat[a1,1:(a2-1)] <- PE[3]*mt1 + gmat[a1,1:(a2-1)]
    }
    if(a2<nallele[l]) 
    {
        gmat[a1,(a2+1):nallele[l]] <- PE[3]*mt1 + gmat[a1,(a2+1):nallele[l]]
    }
    # event 4
    xx <- matrix(0,nallele[l],nallele[l])
    if(a1>1 & a2>1) 
    {
        xx[1:(a1-1),1:(a2-1)] <- PE[4]+xx[1:(a1-1),1:(a2-1)]
    }
    if(a1<nallele[l] & a2<nallele[l]) 
    {
        xx[(a1+1):nallele[l],(a2+1):nallele[l]] <- PE[4]+xx[(a1+1):nallele[l],(a2+1):nallele[l]]
    }
    if(a1>1 & a2<nallele[l]) 
    {
        xx[1:(a1-1),(a2+1):nallele[l]] <- PE[4] + xx[1:(a1- 1),(a2+1):nallele[l]]
    }
    if(a1<nallele[l] & a2>1) 
    {
        xx[(a1+1):nallele[l],1:(a2-1)] <- PE[4] + xx[(a1+1):nallele[l],1:(a2-1)]
    }
    wx <- length(which(xx>0)) 
    gmat <- (xx/wx) + gmat
    #event 7
    xx <- matrix(0,nallele[l],nallele[l])
    if(a2>1) 
    {
        xx[1:(a2-1),1:(a2-1)] <- PE[7]
    }
    if(a2<nallele[l]) 
    {
        xx[(a2+1):nallele[l],(a2+1):nallele[l]] <- PE[7]
    }
    wx <- length(which(xx>0)) 
    gmat <- (xx/wx) + gmat
    #event 8
    xx <- matrix(0,nallele[l],nallele[l])
    if(a1>1) 
    {
        xx[1:(a1-1),1:(a1-1)] <- PE[8]
    }
    if(a1<nallele[l]) 
    {
        xx[(a1+1):nallele[l],(a1+1):nallele[l]] <- PE[8]
    }
    wx <- length(which(xx>0)) 
    gmat <- (xx/wx) + gmat
    gmat <- gmat/sum(gmat)
    
    return(gmat) 
} #end of pgo.func function

#Probability of true genotype (G) given observed genotype (Go):
pg.func <- function(a1,a2,e1,e2,l)
{ #gives true given observed
    pogt <- matrix(0,nallele[l],nallele[l])
    for (t in 1:nallele[l]) #for the tth allele at the lth locus
    { 
        for (y in 1:nallele[l]) #for the yth allele at the lth
        {
            gmat <- pgo.func(t,y,e1,e2,l)
            pogt[t,y] <- gmat[a1,a2] #probability of observed given true genotype ty 
        }
    }
    #divided by sum over all possible genotypes 
    ptgo <- pogt*pgen[[l]]/sum(pogt*pgen[[l]])
    
    return (ptgo) 
}

#Probability of seed dispersal from sampled adults to location of each seedling
dprob.func <- function(us)
{ 
    dprob <- matrix(NA,noff,nadult)
    for (i in 1:noff)
    { 
        for (j in 1:nadult)
        {#d.po is distance of sampled parent to sampled offspring 
            dprob[i,j] <- disp.func(d.po[i,j],us) 
        }
    }
    
    return(dprob) 
}

#Probability of pollen dispersal between sampled adults
pprob.func <- function(up)
{
    pprob <- matrix(NA,nadult,nadult)
    for (j in 1:nadult)
    { 
        for (k in 1:nadult)
        {#d.pp is ditance of sampled parent to sampled parent
            pprob[j,k]<-disp.func(d.pp[j,k],up) 
            if(j==k)
            { #no selfing allowed
                pprob[j,k]<-0
            }
        }
    } 
    return(pprob) 
}


#Pollen expected to go from hypothetical outer tree to outer tree
otop.func <- function(up)
{
    otop <- matrix(0,ldm,ldf) 
    for(j in 1:ldm)
    {
        ppo <- rep(0,ldf) 
        x <- 0

        for(i in 1:ldf)
        {
            x <- dm[j]-df[i] 
            ao <- 0
            ai <- 0

            if (x>=0) 
            {
                ao <- pi*(df[i]^2)
                if(i>1)
                { 
                    ao <- ao-pi*(df[i-1]^2)
                }
            }

            if (x<0) 
            {
                di <- dm[j]
                R <- df[i]
                theta <- 2*((pi/2)-asin(di/R))
                ai <- ((R^2)/2)*(theta-sin(theta)) 
                ao <- pi*(R^2)- ai
                if((R-5)>di)
                {
                    R2 <- df[i-1]
                    theta2 <- 2*((pi/2)-asin(di/R2))
                    ai2 <- ((R2^2)/2)*(theta2-sin(theta2)) 
                    ao2 <- pi*(R2^2)- ai2
                    ao <- ao-ao2
                    ai <- ai-ai2
                } 
            }

            ppo[i]<- wdens*ao*disp.func(df[i],up) #exp pol -> oop moth j from each oop father 
        }#end father-distance loop

    otop[j,] <- ppo #pollen weight to outer distance j from outside 
    }# end mother-distance loop

    return (otop) 
}

#Expected pollen arriving at sampled trees from outside
out.pol.func <- function(up)
{ 
    #ldm = number of hypothetical mothers outside the plot for each side of the plot
    op <- matrix(0,nadult,ldm)
    for(i in 1:nadult)
    {
        for (j in 1:ldm)
        {
            k <- rep(NA,pcr) #pcr - number of corners of plot
            for (t in 1:pcr)
            {
    #disp.func is the 2D-t dispersal function(d,u), where d is distance and u is shape parameter 
    #dist.omf.a is a list where dist.omf.a[[t] is the distance matrix of every adult tree to 
        # the midpoint of each (each = 1,2,...10) hypothetical parent outside of the plot of the  
        # t'th (i=1,2,..4) side of the concentric squares surrounding the plot
    #pollen.f[j,t] is the expected "weight" in the j'th ring outside of the t'th side of the plot 
        #(expected trees * expected weight)
                k[t] <- disp.func( dist.omf.a[[t]][j,i] , up) * pollen.f[j,t]
            } 
        ##op[i,j] is the total probability of pollen reaching the i'th sampled tree's location 
            #from the j'th distance unit outside of the plot
            op[i,j] <- sum(k)
        }#end j loop
    }#end i loop 
    return(op)
}

#Pollen expected to go from sampled adults to hypothetical outside adults
iopol.func <- function(up)
{
    #ldm = number of hypothetical mothers outside the plot for each side of the plot
    iopol <- matrix(0,nadult,ldm)
    for(j in 1:ldm)
    { 
        for(i in 1:nadult)
        {
            k <- rep(NA,pcr) #pcr - number of corners of plot
            for (t in 1:pcr)
            {
    #disp.func is the 2D-t dispersal function(d,u) in units of seeds/m^2, 
        #where d is distance and u is shaape parameter
    #dist.omf.a is a list where dist.omf.a[[i]] is the distance matrix of every adult tree to 
        # the midpoint of each (each = 1,2,...10) hypothetical parent outside of the plot of the  
        # i'th (i=1,2,..4) side of the concentric squares surrounding the plot
    #w[i] is the weight of the i'th adult (by fecundity, size, etc)
    #tree.m[j,t] is the expected number of the trees within the j'th ring outside of the
        #t'th side of the plot (area * trees/area = trees)
    #thus, k[t] is the probability of dispersal from sampled adults to a hypothetical
        #tree outside of the t'th border of the plot
                k[t] <- disp.func( dist.omf.a[[t]][j,i] , up) * w[i] * tree.m[j,t] 
            }
    ##iopol[i,j] is the total probability of pollen from the i'th sampled tree's location 
        #reaching the j'th distance unit outside of the plot
                iopol[i,j] <- sum(k) 
        }
    }

    return(iopol)
}

#Expected seed arriving at seedling plots from outside plot
out.sd.func <- function(us)
{ 
    #ldm = number of hypothetical mothers outside the plot for each side of the plot
    osd <- matrix(0,noff,ldm) 
    for(i in 1:noff)
    { 
        for (j in 1:ldm)
        {
            k <- rep(NA,pcr) #pcr - number of corners of plot
            for (t in 1:pcr)
            {
    #disp.func is the 2D-t dispersal function(d,u) in units of seeds/m^2, 
        #where d is distance and u is shape parameter 
    #dist.omf.o is the distance of hypothetical parents to sampled seedlings
    #seed.m[j,t] is the expected "weight" in the j'th ring outside of the t'th side of the plot
        #(expected trees * expected weight)
    #pld is the area of the plot that the seedling is located within
    #thus, k[t] is the probability of dispersal to the seedling from a parent outside of the 
        #t'th border of the plot
                k[t] <- disp.func( dist.omf.o[[t]][j,i] , us ) * seed.m[j,t] * pld[i] 
            }
            #osd[i,j] is the total probability of a seed reaching the i'th seedling's location
                #from an unsampled tree j distance units outside of the plot
            osd[i,j] <- sum(k) 
        }#end j loop 
    }#end i loop
    
    return(osd) 
}

# Step 3: Calculations depending on above functions

In [ ]:
pogt <- numeric(0) 
refmat <- numeric(0)
for(l in 1:nloci)
{
    m1 <- rep(seq(1,nallele[[l]]),nallele[[l]]) 
    m2 <- numeric(0)
    for(j in 1:nallele[[l]])
    { 
        q <- rep(j,nallele[[l]]) 
        m2 <- c(m2,q)
    }
    m3 <- seq(1,nallele[[l]]*nallele[[l]])
    m <- cbind(m1,m2,m3)
    #create a 3d array (ot) with x =nallele[[l]]^2, y = z = nallele[[l]]:
    ot <- array(0,dim=c(nallele[[l]]*nallele[[l]],nallele[[l]],nallele[[l]])) 
    for(i in 1:nallele[[l]])
    {
        for(j in 1:nallele[[l]])
        {#pgo.func is the probability of observed genotype (Go) given true genotype (G)
            ot[m[,1]==i & m[,2]==j,,] <- pgo.func(i,j,e1,e2,l)
        }
    }# end allele loops
    pogt <- append(pogt,list(ot)) #probabilities of genotypes for each locus
    refmat <- append(refmat,list(m)) 
}# end locus loop"

In [ ]:
#adult to adult distances within plot
d.pp <- matrix(NA,nadult,nadult)
for (i in 1:nadult)
    { 
        for (j in 1:nadult)
        {
            t <- distmat(adult.loc[i,1],adult.loc[i,2],adult.loc[j,1],adult.loc[j,2])
            d.pp[i,j] <- t 
        }
    }

In [ ]:
#adult to offspring distances within plot
d.po <- matrix(NA,noff,nadult) 
for (k in 1:noff)
{
    for (j in 1:nadult)
    {
        t <- distmat(off.loc[k,1],off.loc[k,2],adult.loc[j,1],adult.loc[j,2]) 
        d.po[k,j] <- t
    }
}

In [ ]:
#distance between cornerposts of stand
post.dist <- distmat(pcor[,1],pcor[,2],pcor[,1],pcor[,2])

In [ ]:
pcor

In [ ]:
post.dist

In [ ]:
#length and width of stand as means, for cases in which stand is not perfectly rectangular
ln <- round(mean(c(post.dist[1,2],post.dist[4,3])),digits=1) #length 
wd <- round(mean(c(post.dist[1,4],post.dist[2,3])),digits=1) #width 
edge.length <- c(wd,ln,wd,ln)

In [ ]:
edge.length

In [ ]:
#Find midpoints of stand edges
em1a <- mean(c(pcor[1,1],pcor[2,1])); 
em1b <- mean(c(pcor[1,2],pcor[2,2])) 
em2a <- mean(c(pcor[2,1],pcor[3,1])); 
em2b <- mean(c(pcor[2,2],pcor[3,2])) 
em3a <- mean(c(pcor[3,1],pcor[4,1])); 
em3b <- mean(c(pcor[3,2],pcor[4,2])) 
em4a <- mean(c(pcor[4,1],pcor[1,1])); 
em4b <- mean(c(pcor[4,2],pcor[1,2]))
edgemid <- rbind(c(em1a,em1b),c(em2a,em2b),c(em3a,em3b),c(em4a,em4b))

In [ ]:
#What is the area of the mapped stand?
parea <- wd*ln

In [ ]:
parea == 200**2

In [ ]:
##Approximation for dispersal from outside stand: Consider potential 
#parents out to 100 m from the edge of the stand, at 10 m intervals (both 
#these numbers can be changed, but OD must be equal to or less than the minimum stand edge length).
OD <- 100 
OI <- 10
dm <- seq(OI,OD,by=OI) #seq. of hypothetical mother distances from plot 
ldm <- length(dm) # len = OI
df <- seq(OI,OD,by=OI) # seq. of hypothetical father distances from plot 
ldf <- length(df) # len = OI
e.l <- matrix(0,ldm,4) 
for(i in 1:ldm)
{
    e.l[i,]<-edge.length+(dm[i]*2) #edge length of outer rings 
}

In [ ]:
#What weight attributed to each adult?
weight <- bframe1[ba1,4]
mw <- mean(weight) #average weight

In [ ]:
mw

In [ ]:
#total sampled "weight" 
tw<- sum(weight)
tw

# fix parea
200 x 200 x 12

In [ ]:
#weight or trees per area: 
# wdens <- tw/parea
# tdens <- nadult/parea
wdens <- tw/200*200*12
tdens <- nadult/200*200*12

In [ ]:
#Location of hypothetical out-of-plot parents – at midpoint of each trapezoidal quarter-ring going outward from stand
out.mf <- numeric(0) 
for (k in 1:4) #for each side of a square
{
    hop <- matrix(0,ldm,2) 
    for(i in 1:ldm)
    {
        if(k==1) #for top/north of square
        {
            hop[i,]<- c(edgemid[1,1],edgemid[1,2]+dm[i])
        } 
        if(k==2) #for left/west side of square
        {
            hop[i,]<- c(edgemid[2,1]-dm[i],edgemid[2,2]) 
        }
        if(k==3) #for bottom/south side of square
        {
            hop[i,]<- c(edgemid[3,1],edgemid[3,2]-dm[i])
        } 
        if(k==4) #for right/east side of square
        {
            hop[i,]<- c(edgemid[4,1]+dm[i],edgemid[4,2]) 
        }
    }# end ldm loop
    #out.mf is a list for each side of the sqaure, with [[i]] = a list of len=OI of coordinates of midpoints of outer line to draw
    out.mf <- append(out.mf,list(hop)) 
}

In [ ]:
#Location of hypothetical out-of-plot parents – at midpoint of each trapezoidal quarter-ring going outward from stand
out.mf <- numeric(0) 
for (k in 1:4) #for each side of a square
{
    hop <- matrix(0,ldm,2) 
    for(i in 1:ldm)
    {
        if(k==1) #for top/north of square
        {
            hop[i,]<- c(edgemid[1,1],edgemid[1,2]+dm[i])
        } 
        if(k==2) #for left/west side of square
        {
            hop[i,]<- c(edgemid[2,1]-dm[i],edgemid[2,2]) 
        }
        if(k==3) #for bottom/south side of square
        {
            hop[i,]<- c(edgemid[3,1],edgemid[3,2]-dm[i])
        } 
        if(k==4) #for right/east side of square
        {
            hop[i,]<- c(edgemid[4,1]+dm[i],edgemid[4,2]) 
        }
    }# end ldm loop
    #out.mf is a list for each side of the sqaure, with [[i]] = a list of len=OI of coordinates of midpoints of outer line to draw
    out.mf <- append(out.mf,list(hop)) 
}

In [ ]:
#Distance of hypothetical parents to sampled trees
plot.adult.loc <- cbind(bframe1[ba1,'x'],bframe1[ba1,'y'])

dist.omf.a <- numeric(0) 
for(k in 1:4)
{
    # dx has rows = outs, cols = in parents
    # dx is a distance matrix with dx[i,j] being distance of the ith outside midpoint to the jth adult
    dx <- distmat(plot.adult.loc[,1],plot.adult.loc[,2],out.mf[[k]][,1],out.mf[[k]][,2]) 
    #at the end of the for(loop) dist.omf.a is a list where dist.omf.a[[i]] is the distance matrix of every 
        #adult tree to the midpoint of each (each = 1,2,...10) hypothetical parent outside of the plot 
        #of the i'th (i=1,2,..4) side of the concentric squares surrounding the plot
    dist.omf.a <- append(dist.omf.a,list(dx)) 
}

In [ ]:
#Distance of hypothetical parents to seedlings
plot.off.loc <- cbind(bframe2[bo1,'x'],bframe2[bo1,'y'])

dist.omf.o <- numeric(0)
for(k in 1:4)
{
    dx <- distmat(plot.off.loc[,1],plot.off.loc[,2],out.mf[[k]][,1],out.mf[[k]][,2])
    #rows = outs, cols = offspring 
    dist.omf.o <- append(dist.omf.o,list(dx))
}

In [ ]:
#areas of rectangular rings around plot
a.out.m <- matrix(0,ldm,4) 
for (i in 1:ldm) #for each of the concentric squares
{
    for(e in 1:pcr) #for each side of a square (pcr = 4)
    {
        a.out.m[i,e]<- OI*((edge.length[e]+e.l[i,e])/2)
    }
}

In [ ]:
tree.m <- a.out.m*tdens #expected trees in each ring side  (area * trees/area = trees)
# mw = mean(weight) 
seed.m <- tree.m*mw #expected "weight" in each ring side
#change pollen.f if pollen and seed production are not equally 
    #weighted within single trees
pollen.f = seed.m 

In [ ]:
#Distance of offspring to an edge
#this is not distance to an edge, this is distance to the edge midpoint
d.edge.o <- matrix(NA,noff,4)
for (i in 1:noff)
{ 
    for (j in 1:4)
    {
        t <- distmat(plot.off.loc[i,1],plot.off.loc[i,2],edgemid[j,1],edgemid[j,2])
        d.edge.o[i,j] <- t 
    }
}

In [ ]:
#Distance of adults to an edge
#this is not distance to an edge, this is distance to the edge midpoint
d.edge.a <- matrix(NA,nadult,4)
for (i in 1:nadult)
{ 
    for (j in 1:4)
    {
        t <- distmat(plot.adult.loc[i,1],plot.adult.loc[i,2],edgemid[j,1],edgemid[j,2])
        d.edge.a[i,j] <- t 
    }
}

# Step 4: Functions for comparing genotypes

In [ ]:
x = freq[3,1:nallele[3]]

In [ ]:
y = rep(1,nallele[3])
len(x) == len(y)

In [ ]:
len(x)

In [ ]:
len(y)

In [ ]:
#Calculate probability of offspring genotype given parental genotypes
nmat.func <- function(pg1,pg2,pg1.b,pg2.b,la) #la = lth allele from a for(loop)
{ 
    if (pg1[1,la]>0)
    { #a genotype is observed in mother
        #calculate probabilty of true genotype given observed genotype
        ptgo1 <- pg.func(pg1[1,la],pg1[2,la],e1,e2,la) #pg.func = prob of true genotype (G) given observed genotype (Go)
        p1a <- apply(ptgo1,1,sum) #probs for true allele 1 given allele 1
        p1b <- apply(ptgo1,2,sum) #probs for true allele 2 given allele 1
        if(pg1.b[1,la]>0)
        { #if locus regenotyped
            ptgo1.r <- pg.func(pg1.b[1,la],pg1.b[2,la],e1,e2,la)
            p1a.r <- apply(ptgo1.r,1,sum)
            p1b.r <- apply(ptgo1.r,2,sum)
            p1a <- p1a*p1a.r; p1b <- p1b*p1b.r
        } #end 2nd
    } #end 'if observed'
    if (pg1[1,la]==0)
    {#if no genotype observed in mother
        if (la>4) # 4 = Nnuclear
        {
            #if we're considering chloroplast markers
            p1a <- rep(1,nallele[[la]]) #we want the probability of chloroplast markers to be independent of mother
            p1b <- rep(1,nallele[[la]]) #we want the probability of chloroplast markers to be independent of mother
        }         
        else
        {
            p1a <- freq[la,1:nallele[[la]]]
            p1b <- freq[la,1:nallele[[la]]]   
        }
    }
    if (pg2[1,la]>0)
    { #a genotype is observed for father
        ptgo2 <- pg.func(pg2[1,la],pg2[2,la],e1,e2,la)
        p2a <- apply(ptgo2,1,sum) #probs for allele 1 
        p2b <- apply(ptgo2,2,sum) #probs for allele 2
        if(pg2.b[1,la]>0)
        { #if locus regenotyped
            ptgo2.r <- pg.func(pg2.b[1,la],pg2.b[2,la],e1,e2,la)
            p2a.r <- apply(ptgo2.r,1,sum)
            p2b.r <- apply(ptgo2.r,2,sum)
            p2a <- p2a*p2a.r
            p2b <- p2b*p2b.r
        } #end 2nd
    }# end 'if observed'
    if (pg2[1,la]==0)
    { #if no genotype observed for father
        p2a <- freq[la,1:nallele[[la]]]
        p2b <- freq[la,1:nallele[[la]]] 
    }
    #probability of all possible offspring genotypes
    p1a.m1 <- matrix(p1a,nallele[[la]],nallele[[la]])
    p2a.m1 <- matrix(p2a,nallele[[la]],nallele[[la]],byrow=T) 
    p1b.m1 <- matrix(p1b,nallele[[la]],nallele[[la]])
    p2b.m1 <- matrix(p2b,nallele[[la]],nallele[[la]],byrow=T)
    h1.b <- (p1a.m1*p2a.m1)/8
    h2.b <- (t(p1a.m1)*t(p2a.m1))/8 
    h3.b <- (p1a.m1*p2b.m1)/8 
    h4.b <- (t(p1a.m1)*t(p2b.m1))/8 
    h5.b <- (p1b.m1*p2a.m1)/8 
    h6.b <- (t(p1b.m1)*t(p2a.m1))/8 
    h7.b <- (p1b.m1*p2b.m1)/8 
    h8.b <- (t(p1b.m1)*t(p2b.m1))/8
    nmat <- h1.b+h2.b+h3.b+h4.b+h5.b+h6.b+h7.b+h8.b

    return(nmat) 
}#end function

In [ ]:
#Probability of observing the given offspring genotype given all possible true genotypes:
omat.func <- function(og,og.b,la) #la = lth allele from a for(loop)
{ #prob of observing off gen, given true 
    if(og[1,la]>0)
    { #if a genotype observed
        if(og.b[1,la]==0)
        { #if not regenotyped 
            omat <- matrix(0,nallele[[la]],nallele[[la]])
            for(x in 1:nallele[[la]])
            { 
                for (y in 1:nallele[[la]])
                {
                    u <- pogt[[la]][refmat[[la]][,1]==x & refmat[[la]][,2]==y,,]
                    omat[x,y] <- u[og[1,la],og[2,la]] 
                }
            }
        } #end no 2nd
        if(og.b[1,la]>0)
        { #if regenotyped
            omat <- matrix(0,nallele[[la]],nallele[[la]]) 
            omat.r <- matrix(0,nallele[[la]],nallele[[la]])
            for(x in 1:nallele[[la]])
                { 
                    for (y in 1:nallele[[la]])
                    {
                        u <- pogt[[la]][refmat[[la]][,1]==x & refmat[[la]][,2]==y,,] 
                        omat[x,y] <- u[og[1,la],og[2,la]]
                        omat.r[x,y] <- u[og.b[1,la],og.b[2,la]]
                    }
                }
            omat <- omat*omat.r
        } #end 2nd
    }# end 'if observed'
    if (og[1,la]==0) #if not genotyped
    { 
        omat <- matrix(1,nallele[[la]],nallele[[la]]) 
        omat <- omat/sum(omat)
    }
    return(omat) 
}

# way way way way too slow, multiprocessing to the rescue

In [ ]:
dir <- function(dir,name){
    return(paste(dir,name,sep=""))
}

In [ ]:
DIR = '/home/lindb/teakettle/data/moran/dependencies/'

In [ ]:
#save env
for (obj in ls()){
    saveRDS(get(obj),dir(DIR,sprintf("%s.RDS",obj)))
}

In [ ]:
#make sure I can load back in
dname = '/home/lindb/teakettle/data/moran/dependencies/'
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %s file\n",f))
        fname = paste(dname,f,sep="")
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(fname))    
    }
}

In [ ]:
nadult

In [ ]:
noff

## go to [04_multiprocess_parentage.ipynb](./04_multiprocess_parentage.ipynb)

# To do
1. check on pcor
1. check on pcr
1. adult.loc
1. off.loc
1. e1,e2
1. dispersal priors

# for Moran
1. pg.func in nmat.func loop
1. amat[[k]] in (if(==j % i<(nadult+1))
1. amat sum at end of loop

In [ ]:
amat <- numeric(0)
for (k in 1:noff)
{
    a <- matrix(0,(nadult+1),(nadult+1)) 
    amat <- append(amat,list(a))
}

In [ ]:
remove(delete)

In [ ]:
delete <- numeric(0)
t <- system.time(for (k in 1:noff){
    a <- matrix(0,(nadult+1),(nadult+1))
    delete <- append(delete,list(a))
})
t

In [ ]:
system.time(saveRDS(delete,'~/teakettle/delete_amat_time.RDS'))

In [ ]:
file.info('~/teakettle/delete_amat_time.RDS')

In [ ]:
((558*558)/2) - 558/2

In [ ]:
29.402*155403 #seconds

In [ ]:
4569159.006/60 #minutes

In [ ]:
76152.6501/60 #hours

In [ ]:
1269.210835/24 #days

In [ ]:
52.8837847916667/160

In [ ]:
0.330523654947917*24 #difference of ~8 hours